## Ensemble learning for activity recognition

### 1. Set libraries

In [204]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)
import keras as keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from keras import optimizers
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from random import randint
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat
from keras.layers import GRU, LSTM, Activation, Bidirectional
from keras.layers import Dense, Embedding
from keras.layers import TimeDistributed
from keras.layers import Concatenate, Flatten 
from keras.layers import Dense, concatenate
import time
import matplotlib.pyplot as plt
# stacked generalization with linear meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from numpy import dstack
from attention import Attention

# univariate cnn lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import SpatialDropout1D

### 1. Load the data

In [205]:
# load dataset with pandas
dataframe1 = pd.read_csv('data/input_v2/Presentation_activity.csv', sep=',', header=None)
dataframe2 = pd.read_csv('data/input_v2/Chatting_activity.csv', sep=',', header=None)
dataframe3 = pd.read_csv('data/input_v2/Discussion_activity.csv', sep=',', header=None)
dataframe4 = pd.read_csv('data/input_v2/GroupStudy_activity.csv', sep=',', header=None)

# Show how many episodes 
#print(len(dataframe1))
#print(len(dataframe2))
#print(len(dataframe3))
#print(len(dataframe4))

#Combine all dataframe
dataframe = dataframe1.append(dataframe2,ignore_index=True)
dataframe = dataframe.append(dataframe3,ignore_index=True)
dataframe = dataframe.append(dataframe4,ignore_index=True)

#Change NaN data to 0
dataframe = dataframe.fillna(0)
dataset = dataframe.values

#288, 6
#Seperate input and do one-hot encoding
#X=to_categorical(dataset)
X=dataset
print("The shape of X:", X.shape)

# Construct Y label
arr = [] #  empty regular list
for i in range(len(dataframe1)):
    arr.append(0*np.ones((1)))
for i in range(len(dataframe2)):
    arr.append(1*np.ones((1)))
for i in range(len(dataframe3)):
    arr.append(2*np.ones((1)))
for i in range(len(dataframe4)):
    arr.append(3*np.ones((1)))
np_array = np.array(arr)  # transformed to a numpy array
y_df = pd.DataFrame({'': np_array[:, 0]})

Y=to_categorical(y_df)
#Y=y_df
print("The shape of Y:", Y.shape)


# Divide train and test data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

The shape of X: (340, 724)
The shape of Y: (340, 4)


### 2. Train DeepLSTM model

In [206]:
from keras.layers import Bidirectional
length=724
model1 = Sequential()
model1.add(Embedding(length, 100, input_length=X.shape[1]))
model1.add(SpatialDropout1D(0.2))
model1.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model1.add(Dense(4, activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0


In [ ]:
#tf.debugging.set_log_device_placement(True)
epochs = 20
batch_size = 64
history = model1.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)
# evaluate LSTM
# evaluate model
accr = model1.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
print(accr)

plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

yhat1=model1.predict(X_test,verbose=0)
argmax(yhat1, axis=1)

C:\Users\Anaconda3\envs\tf2.0-gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 244 samples, validate on 28 samples
Epoch 1/20
Executing op __inference_keras_scratch_graph_262206 in device /job:localhost/replica:0/task:0/device:GPU:0


In [ ]:
model2 = Sequential()
model2.add(Embedding(length, 100, input_length=X.shape[1]))
model2.add(SpatialDropout1D(0.2))  
model2.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(4, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#tf.debugging.set_log_device_placement(True)
batch_size = 64
# Divide train and test data
#X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)
history = model2.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)
# evaluate LSTM
# evaluate model
accr = model2.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
print(accr)

plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

yhat2=model2.predict(X_test,verbose=0)
argmax(yhat2, axis=1)

In [ ]:
model3 = Sequential()
model3.add(Embedding(length, 100, input_length=X.shape[1]))
model3.add(SpatialDropout1D(0.2))  
model3.add(GRU(100, dropout=0.2, recurrent_dropout=0.2))
model3.add(Dense(4, activation='softmax'))
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#tf.debugging.set_log_device_placement(True)
epochs = 20
batch_size = 64
# Divide train and test data
#X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)
history = model3.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)
# evaluate LSTM
# evaluate model
accr = model3.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
print(accr)

plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

yhat3=model3.predict(X_test,verbose=0)
argmax(yhat3, axis=1)

In [ ]:
model4 = Sequential()
model4.add(Embedding(length, 100, input_length=X.shape[1]))
model4.add(SpatialDropout1D(0.2))  
model4.add(Bidirectional(GRU(100, dropout=0.2, recurrent_dropout=0.2)))
model4.add(Dense(4, activation='softmax'))
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#tf.debugging.set_log_device_placement(True)
epochs = 20
batch_size = 64
# Divide train and test data
#X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)
history = model4.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)
# evaluate LSTM
# evaluate model
accr = model4.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
print(accr)

plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

yhat4=model4.predict(X_test,verbose=0)
argmax(yhat4, axis=1)

In [ ]:
n_features = 1
n_seq = 4
n_steps = 181
# define model
model5 = Sequential()
model5.add(TimeDistributed(Conv1D(filters=4, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
model5.add(TimeDistributed(MaxPooling1D(pool_size=4)))
model5.add(TimeDistributed(Flatten()))
model5.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model5.add(Dense(4, activation='softmax'))
model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 20
batch_size = 64
# fit model
X_train1 = X_train.reshape((X_train.shape[0], n_seq, n_steps, n_features))
model5.fit(X_train1, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)
# demonstrate prediction    
X_test1 = X_test.reshape((X_test.shape[0], n_seq, n_steps, n_features))
accr = model5.evaluate(X_test1, Y_test, batch_size=batch_size, verbose=0)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
yhat5 = model5.predict(X_test1, verbose=0)
print(yhat5)

In [ ]:
from keras.layers import ConvLSTM2D
n_features = 1
n_seq = 4
n_steps = 181
# define model
model6 = Sequential()
model6.add(ConvLSTM2D(filters=4, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model6.add(TimeDistributed(Flatten()))
model6.add(Dense(4, activation='softmax'))
model6.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model6.summary()

In [ ]:
epochs = 20
batch_size = 64
# fit model
X_train2 = X_train.reshape((X_train.shape[0], n_seq, 1, n_steps, n_features))
Y_train2 = Y_train.reshape((Y_train.shape[0], 1, Y_train.shape[1]))
model6.fit(X_train2, Y_train2, epochs=epochs, batch_size=batch_size,validation_split=0.1)
# demonstrate prediction    
X_test2 = X_test.reshape((X_test.shape[0], n_seq, 1, n_steps, n_features))
Y_test2 = Y_test.reshape((Y_test.shape[0], 1, Y_test.shape[1]))
accr = model6.evaluate(X_test2, Y_test2, batch_size=batch_size, verbose=0)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
yhat6 = model6.predict(X_test2, verbose=0)
print(yhat6)

In [ ]:
from os import makedirs
 
# create directory for models
#makedirs('models')

# save model
filename = 'models/model_1.h5'
model1.save(filename)
print('>Saved %s' % filename)

filename = 'models/model_2.h5'
model2.save(filename)
print('>Saved %s' % filename)

filename = 'models/model_3.h5'
model3.save(filename)
print('>Saved %s' % filename)

filename = 'models/model_4.h5'
model4.save(filename)
print('>Saved %s' % filename)

filename = 'models/model_5.h5'
model5.save(filename)
print('>Saved %s' % filename)

filename = 'models/model_6.h5'
model6.save(filename)
print('>Saved %s' % filename)

In [ ]:
res_list = [(yhat1[i] + yhat2[i]) for i in range(len(yhat2))] 
res_list
r1=argmax(res_list, axis=1)
print(r1)
r2=argmax(Y_test, axis=1)
print(len(r2))
count=0
for j in range(len(r2)):
   if r1[j]==r2[j] :
        count+=1
print(count/len(r2))

In [ ]:
yhat6 = np.reshape(yhat6,(len(yhat6),4))
res_list = [(yhat1[i] + yhat2[i] + yhat3[i]+yhat5[i]+yhat6[i]) for i in range(len(yhat2))] 
res_list
r1=argmax(res_list, axis=1)
print(r1)
r2=argmax(Y_test, axis=1)
print(len(r2))
count=0
for j in range(len(r2)):
   if r1[j]==r2[j] :
    count=count+1
print(count/len(r2))

In [ ]:
true_rs = 0*len(Y_test)
true_rs = argmax(Y_test, axis=1)
print(true_rs)
pre_rs = np.full(( len(Y_test),4),0)
rs1 =  argmax(yhat1, axis=1)
rs2 =  argmax(yhat2, axis=1)
rs3 =  argmax(yhat3, axis=1)
rs4 =  argmax(yhat4, axis=1)
rs5 =  argmax(yhat5, axis=1)
rs6 =  argmax(yhat6, axis=1)

i=0
for i in range(len(Y_test)):
    pre_rs[i][rs1[i]]+= 1
    #pre_rs[i][rs2[i]]+= 1
    #pre_rs[i][rs3[i]]+= 1
    pre_rs[i][rs4[i]]+= 1
    pre_rs[i][rs5[i]]+= 1
    pre_rs[i][rs6[i]]+= 1 
    
count=0
rs_final = argmax(pre_rs, axis=1)
print(rs_final)
for j in range(len(true_rs)):
   if true_rs[j]== rs_final[j] :
        count+=1
print(count/len(true_rs))    

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from itertools import product

# normalize a vector to have unit norm
def normalize(weights):
	# calculate l1 vector norm
	result = norm(weights, 1)
	# check for a vector of all zeros
	if result == 0.0:
		return weights
	# return normalized vector (unit norm)
	return weights / result
r2=argmax(Y_test, axis=1)
members=[model1,model2,model3,model4,model5,model6]
yhats=[yhat1, yhat2, yhat3, yhat4, yhat5, yhat6]
# define weights to consider
w = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
best_score, best_weights = 0.0, None
for weights in product(w, repeat=len(members)):
	# skip if all weights are equal
	if len(set(weights)) == 1:
		continue
# hack, normalize weight vector
	weights = normalize(weights)
# evaluate weights
	yhats =  array(yhats)
	summed =  tensordot(yhats, weights, axes=((0),(0)))
	result = argmax(summed, axis=1)
	score =  accuracy_score(r2, result)
	if score > best_score:
		best_score, best_weights = score, weights
		print('>%s %.3f' % (best_weights, best_score))